### List of usable APIs - https://api.census.gov/data.html
### List of variables in the API currently used - https://api.census.gov/data/2021/pdb/tract/variables.html

In [3]:
import requests
import json
import io
import pandas as pd

In [4]:
def fetchTractDataVars():
    """
    Fetches the variable names, descriptions, etc found within the census tract data
    """
    
    # If you want to change the api used, change this link to what the *variables* link sends to you (with .json instead of .html)
    varsLink = "https://api.census.gov/data/2021/pdb/tract/variables.json"
    return json.loads(requests.get(varsLink).text)

In [ ]:
def fetchTractData(variables, year = 2021):
    """
    Generates a request string and requests census API for desired variables. Output is returned as pandas dataframe
    Parameters:
        variables: list of variable names you want to include
    """
    
    #Again, if you want to change the API used, change the link accordingly. Additionally, the format may or may not change
    #with different APIs, so look out for that.
    base = "https://api.census.gov/data/{}/pdb/tract".format(year)
    forTract = "&for=tract:*"
    state = "&in=state:39"
    
    #Generating get string for all requested variables
    get = ''
    for var in variables:
        get += var + ','
    get = "?get={}".format(get[:-1])
    reqString = base + get + forTract + state

    #Setting column names to first row vals
    df = pd.DataFrame(json.loads(requests.get(reqString).text))
    df.rename(columns = df.iloc[0], inplace = True)
    df = df.drop(df.index[0])
    
    #Removes duplicate column names
    return df.loc[:,~df.columns.duplicated()]

In [6]:
# Example of how we utilized the API - 
# Include all variables that you want to use

#Tot_Population :       Total population
#Civ_unemp_16plus :     Unemployed population that is 16+
#ENG_VW :               People 14+ who either don't speak english or don't speak english very well
#HHD_w_Broadband:       Has broadband internet access
#Med_HHD_Inc:           Median income access
#MrdCple_Fmly_HHD:      Married couple family households
#No_Health_Ins:         Number of people without health insurance
#pct_Tot_Occp_Units:    Percentage of occupied housing units
#GIDTR:                 Funny name for census tract, love that it's not just 'tract'
#Aggr_House_Value:      Aggregate (total) housing unit value in dollars
#Tot_Housing_Units:     Total # of housing units in ACS survey

df = fetchTractData(['state', 'County_name', 'Tot_Population_ACS_15_19',
                     'Civ_unemp_16plus_ACS_15_19', 'ENG_VW_ACS_15_19', 'GIDTR',
                     'HHD_w_Broadband_ACS_15_19', 'Med_HHD_Inc_ACS_15_19',
                     'Median_Age_ACS_15_19', 'MrdCple_Fmly_HHD_CEN_2010', 
                     'No_Health_Ins_ACS_15_19', 'pct_Tot_Occp_Units_ACS_15_19',
                     'Aggr_House_Value_ACS_15_19', 'Tot_Housing_Units_ACS_15_19'])
df = df.drop(['county', 'tract'], axis = 1)